In [ ]:
import geopandas as gpd
import pandas as pd
from shapely import geometry
from skimage import measure
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.spatial import (
    Voronoi,
    voronoi_plot_2d,
    Delaunay,
    delaunay_plot_2d,
    cKDTree
)
import numpy as np
import math, time
import interpolators
import itertools

plt.rcParams["figure.figsize"] = 30, 20
plt.rcParams["font.size"] = 20
plt.rcParams["axes.titlesize"] = 50
plt.rcParams["axes.titlepad"] = 80

def get_polygons_per_zone_plt(xnew, ynew, interpolated_values, zones):
    fig, ax = plt.subplots()
    contour = ax.contourf(xnew, ynew, interpolated_values, zones, cmap="winter_r")
    plt.close()

    polygons_per_zone = []

    for col in contour.collections:
        zone_polygons = []
        # Loop through all polygons that have the same intensity level
        for contour_path in col.get_paths():

            # Create the polygon for this intensity level
            # The first polygon in the path is the main one, the following ones are "holes"
            poly = None
            for idx, poly_coords in enumerate(contour_path.to_polygons()):
                poly_coords = np.array(poly_coords)
                x = poly_coords[:, 0]
                y = poly_coords[:, 1]

                new_shape = geometry.Polygon(
                    [(point[0], point[1]) for point in zip(x, y)]
                )

                if idx == 0:
                    poly = new_shape
                else:
                    # Remove the holes if there are any
                    poly = poly.difference(new_shape)
                    # Can also be left out if you want to include all rings

            if poly is not None:
                zone_polygons.append(poly)
        polygons_per_zone.append(zone_polygons)
    return polygons_per_zone

def get_polygons_per_zone(xnew, ynew, interpolated_values, zones):
    xmin = np.min(xnew)
    xmax = np.max(xnew)
    ymin = np.min(ynew)
    ymax = np.max(ynew)
    scale_x = lambda x: xmin + (xmax-xmin)/len(xnew)*(x+0.5)
    scale_y = lambda y: ymin + (ymax-ymin)/len(ynew)*(y+0.5)

    polygons_per_zone = []

    # Iterate in reverse to go from most inner zones to outer zones
    # Makes it easier for hole detections
    for zone, zone_limit in enumerate(zones[::-1]):
        contours = measure.find_contours(interpolated_values, zone_limit)
        contour_polygons = list(map(lambda c: geometry.Polygon(zip(scale_x(c[:, 1]), scale_y(c[:, 0]))), contours))
        
        previous_polygons = list(itertools.chain(*polygons_per_zone))
        zone_polygons = []
        holes = []

        for p1 in contour_polygons:
            if p1 in holes:
                continue

            # Check for holes in this current contour
            for p2 in contour_polygons:
                if p1 == p2:
                    continue

                if p1.contains(p2):
                    p1 = p1.difference(p2)
                    holes.append(p2)
            
            # Check if inner contours are holes in current polygon
            for p2 in previous_polygons:
                if p1.contains(p2):
                    p1 = p1.difference(p2)
                    holes.append(p2)
            
            zone_polygons.append(p1)
        polygons_per_zone.append(zone_polygons)
    # Reverse again to return polygons in same order as input zones
    return polygons_per_zone[::-1]

def plot_polygons(polygons):
    polygon_df = gpd.GeoDataFrame()
    for polygon in polygons:
        temp_df = gpd.GeoDataFrame({"geometry": [polygon]})
        polygon_df = pd.concat([polygon_df, temp_df])
    polygon_df.plot()

def get_cmap_colors(cmap_name, n, rgb=True):
    cmap = cm.get_cmap(cmap_name, n)    # PiYG

    colors = []
    for i in range(cmap.N):
        rgb_values = cmap(i)[:3] # will return rgba, we take only first 3 so we get rgb
        if rgb:
            colors.append(",".join(list(map(str,rgb_values))))
        else:
            colors.append(mpl.colors.rgb2hex(rgb_values))
    return colors

external_crs = "EPSG:4326"
internal_crs = "EPSG:3068"
berlin_districts = gpd.read_file("../shared/berlinDistricts.geojson")
# measurements = gpd.read_file("raw-test/data_2020-02-12T14-00-00.geojson")
measurements = gpd.read_file("meeting-test/data_2020-03-02T22-00-00.geojson")
# measurements = gpd.read_file("meeting-test/data_2020-03-02T03-00-00.geojson")

berlin_districts = berlin_districts.to_crs(internal_crs)
measurements = measurements.to_crs(internal_crs)

x = np.array(measurements.geometry.x)
y = np.array(measurements.geometry.y)
values = np.array(measurements.value)
points = np.column_stack((x, y))

xmin, ymin, xmax, ymax = measurements.total_bounds
size = 300  # grid cell size in meters
xnew = np.linspace(xmin, xmax, int((xmax - xmin) / size))
ynew = np.linspace(ymin, ymax, int((ymax - ymin) / size))
zones = [0,20,40, 100, 200, 400]

In [ ]:
# interpolated_values = interpolators.nearest_neighbor(xnew, ynew, points, values)
# interpolated_values = interpolators.linear_barycentric(xnew, ynew, points, values)
# interpolated_values = interpolators.natural_neighbor(xnew, ynew, points, values)
# interpolated_values = interpolators.discrete_natural_neighbor(xnew, ynew, points, values)
interpolated_values = interpolators.inverse_distance_weighting(xnew, ynew, points, values)
# interpolated_values = interpolators.rbf(xnew, ynew, points, values)
# interpolated_values = interpolators.radial_base_function(xnew, ynew, points, values, function="linear")

print(interpolated_values)

In [ ]:
start = time.time()
# polygons_per_zone = get_polygons_per_zone_plt(xnew, ynew, interpolated_values, zones)
polygons_per_zone = get_polygons_per_zone(xnew, ynew, interpolated_values, zones)
print(time.time()-start)

for x in polygons_per_zone:
    print(len(x))

# print(polygons_per_zone)
# plot_polygons(polygons_per_zone[1])

In [ ]:
# Plot Berlin Boundaries with Measurements

fig, ax = plt.subplots()
ax.set_title("Berlin Districts with Measurements")
berlinPlot = berlin_districts.boundary.plot(ax=ax, edgecolor="black")
measurements.plot(ax=ax, column="value", legend=True)

In [ ]:
# Plot Berlin boundaries with interpolation grid

fig, ax = plt.subplots()
ax.set_title("Berlin Districts with Interpolation Grid")
berlin_districts.boundary.plot(ax=ax, edgecolor="black")
xx, yy = np.meshgrid(xnew,ynew)
ax.scatter(xx, yy, s=1)

In [ ]:
# Plot Berlin with Voronoi diagram

voronoi = Voronoi(points)

fig, ax = plt.subplots()
ax.set_title("Berlin Districts with Voronoi Diagram")
berlin_districts.boundary.plot(ax=ax, edgecolor="black")
voronoi_plot_2d(voronoi, ax=ax, show_vertices=False, show_points=True, line_colors='orange')
fig.savefig("voronoi.png")

In [ ]:
# Plot Berlin with Delauny diagram

delauny = Delaunay(points)

fig, ax = plt.subplots()
ax.set_title("Berlin Districts with Delauny Diagram")
berlin_districts.boundary.plot(ax=ax, edgecolor="black")
delaunay_plot_2d(delauny, ax=ax)

In [ ]:
from sklearn.preprocessing import scale

fix, ax = plt.subplots()
ax.hist(values)
ax.hist(scale(values))

In [ ]:
# Variogram Cloud

from scipy.spatial.distance import pdist, squareform

p_distances = pdist(points)

v_distances = 1/2 * (pdist(values.reshape(-1, 1)) ** 2)
fig, ax = plt.subplots()
ax.scatter(p_distances, v_distances)
ax.set_xlabel("h (lag)", fontsize=20)
ax.set_ylabel(r'$\gamma(h)$', fontsize=20)

In [ ]:
# Histogram of value distances

fig, ax = plt.subplots()
ax.hist(v_distances)

In [ ]:
# Experimental Variogram

bins = 50
n, bin_edges = np.histogram(p_distances, bins=bins)
summed_distances_per_bin, bin_edges = np.histogram(p_distances, bins=bins, weights=v_distances)
mean = summed_distances_per_bin / n

fig, ax = plt.subplots()
ax.scatter((bin_edges[1:] + bin_edges[:-1])/2, mean)

In [ ]:
# Interpolated grid points in color

grid = np.meshgrid(xnew, ynew)
new_points = np.reshape(grid, (2, -1)).T

fig, ax = plt.subplots()
scatter = ax.scatter(new_points[:,0], new_points[:,1], s=5, c=interpolated_values.ravel(), cmap="Reds")
scatter.cmap.set_under("w")
scatter.set_clim(zones[1])

In [ ]:
# Colored grid with interpolation as image

fig, ax = plt.subplots()
img = ax.imshow(interpolated_values, origin="lower", cmap="Reds", extent=[xmin, xmax, ymin, ymax])
img.cmap.set_under("w")
img.set_clim(zones[1])

In [ ]:
# Contours using matplotlib

fig, ax = plt.subplots()

img = ax.imshow(interpolated_values, origin="lower", cmap="Reds", extent=[xmin, xmax, ymin, ymax])
img.cmap.set_under("w")
img.set_clim(zones[1])

contour = ax.contour(xnew, ynew, interpolated_values, zones)
contour.cmap.set_under("w")
contour.set_clim(zones[1])
berlin_districts.boundary.plot(ax=ax, edgecolor="black")

In [ ]:
# Contours using skimage and marching squares

from skimage import measure

fig, ax = plt.subplots()

berlin_districts.boundary.plot(ax=ax, edgecolor="black")

img = ax.imshow(interpolated_values, origin="lower", cmap="Reds", extent=[xmin, xmax, ymin, ymax])
img.cmap.set_under("w")
img.set_clim(zones[1])

scale_x = lambda x: xmin + (xmax-xmin)/len(xnew)*(x+0.5)
scale_y = lambda y: ymin + (ymax-ymin)/len(ynew)*(y+0.5)

colors = get_cmap_colors("viridis", len(zones))
for i, zone in enumerate(zones):
    contours = measure.find_contours(interpolated_values, zone)
    color = tuple(map(float, colors[i].split(",")))
    for n, contour in enumerate(contours):
        ax.plot(scale_x(contour[:, 1]), scale_y(contour[:, 0]), linewidth=1.5, color=color)

In [ ]:
import collections, random
from scipy import interpolate
from scipy.spatial.distance import cdist

from pykrige.ok import OrdinaryKriging
from pykrige.uk import UniversalKriging

def ordinary_kriging(x, y, points, value): 
    
    ok_interpolator = OrdinaryKriging(points[:, 0], points[:, 1], values, variogram_model='linear',
                     verbose=True, enable_plotting=True)
    xx, yy = np.meshgrid(x, y)
    ok_interpolator.display_variogram_model()
    # result = ok_interpolator.execute('grid', x, y)
    print(result[0])
    print(np.min(result[0]))
    print(np.max(result[0]))
    return result[0]

def spline(x, y, points, values):
    xx, yy = np.meshgrid(x, y)
    point_matrix = np.dstack((xx, yy))
    tck = interpolate.bisplrep(points[:,0], points[:,1], values)
    grid_values = interpolate.bisplev(xnew, ynew, tck).T
    print(grid_values.shape)
    return grid_values

from sklearn.gaussian_process import GaussianProcessRegressor, GaussianProcessClassifier
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from geostatsmodels import utilities, kriging, variograms, model, geoplot
from geostatsmodels.kriging import krige
from scipy.stats import norm

def kriging(x, y, points, values):
    gpr = GaussianProcessRegressor(normalize_y=True)
    gpr.fit(points, values)

    xx, yy = np.meshgrid(x, y)
    new_points = np.column_stack([xx.flatten(), yy.flatten()])
    result = gpr.predict(new_points)
    print(new_points.shape)
    print(np.min(result))
    print(np.max(result))
    # return result.reshape(y.shape[0], x.shape[0])


    x_point_dup = [
        item for item, count in collections.Counter(points[:, 0]).items() if count > 1
    ]
    y_point_dup = [
        item for item, count in collections.Counter(points[:, 1]).items() if count > 1
    ]
    fixed_x = list(
        map(
            lambda x: x + random.uniform(-0.00001, 0.00001) if x in x_point_dup else x,
            points[:, 0],
        )
    )
    fixed_y = list(
        map(
            lambda y: y + random.uniform(-0.00001, 0.00001) if y in y_point_dup else y,
            points[:, 1],
        )
    )

    fixed_points = np.column_stack((fixed_x, fixed_y))


    nugget = 50
    lags = np.arange(nugget, 60000, 5000)
    sill = np.var(values)
    P = np.array(list(zip(fixed_points[:,0], fixed_points[:, 1], values)))
    geoplot.semivariogram(P, lags, nugget)
    svm = model.semivariance(model.spherical, (4000, sill))
    # geoplot.semivariogram(P, lags, nugget, model=svm)
    covfct = model.covariance(model.spherical, (4000, sill))
    # kriging.simple(P, covfct, pt, N=6)
    est, kstd = krige(P, covfct, new_points, 'simple', N=6)


start = time.time()

# interpolated_values = interpolators.nearest_neighbor(xnew, ynew, points, values)
# interpolated_values = rbf(xnew, ynew, points, values)
# interpolated_values = kriging(xnew, ynew, points, values)
# interpolated_values = ordinary_kriging(xnew, ynew, points, values)
# interpolated_values = interpolators.linear_barycentric(xnew, ynew, points, values)
# interpolated_values = interpolators.natural_neighbor(xnew, ynew, points, values)
# interpolated_values = interpolators.discrete_natural_neighbor(xnew, ynew, points, values)
interpolated_values = interpolators.inverse_distance_weighting(xnew, ynew, points, values)

# interpolated_values = np.array([list(map(getMeasurementValue, row)) for row in interpolated_values])

print(interpolated_values.shape)

xx, yy = np.meshgrid(xnew, ynew)
print(interpolated_values.ravel().shape)
# plt.scatter(xx, yy, interpolated_values)

end = time.time()
print(end - start)